In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

### Loading environment data

In [36]:
env = pd.read_csv('/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/environmental_features.csv')
env.head(10)

,YEAR,LOC,X04_PRCP,X05_PRCP,X06_PRCP,X07_PRCP,X08_PRCP,X09_PRCP,X10_PRCP,X04_TAVG,...,phh2o_15_30cm,phh2o_30_60cm,phh2o_60_100cm,phh2o_100_200cm,soc_0_5cm,soc_5_15cm,soc_15_30cm,soc_30_60cm,soc_60_100cm,soc_100_200cm
0,2001,SDEP,93.100000,80.800000,49.200000,97.400000,55.600000,89.400000,19.300000,9.098235,...,71.0,74.0,78.0,79.0,292.0,190.0,124.0,74.0,46.0,27.0
1,2002,SDEP,77.500000,74.400000,54.700000,46.300000,180.700000,23.900000,67.900000,9.541765,...,71.0,74.0,78.0,79.0,292.0,190.0,124.0,74.0,46.0,27.0
2,2003,SDEP,80.300000,96.600000,162.200000,30.500000,19.100000,197.100000,17.000000,9.234118,...,71.0,74.0,78.0,79.0,292.0,190.0,124.0,74.0,46.0,27.0
3,2004,SDEP,61.800000,157.000000,80.800000,40.800000,43.200000,118.900000,50.041176,9.372353,...,71.0,74.0,78.0,79.0,292.0,190.0,124.0,74.0,46.0,27.0
4,2005,SDEP,103.900000,92.700000,165.200000,84.900000,76.700000,136.700000,18.500000,10.726471,...,71.0,74.0,78.0,79.0,292.0,190.0,124.0,74.0,46.0,27.0
5,2006,IALI,100.700000,56.400000,20.600000,128.700000,166.600000,92.070588,67.276471,11.500000,...,66.0,68.0,70.0,76.0,305.0,235.0,160.0,107.0,42.0,19.0
6,2007,IALI,106.711765,141.900000,113.005882,116.170588,116.900000,93.247059,61.994118,10.117647,...,66.0,68.0,70.0,76.0,305.0,235.0,160.0,107.0,42.0,19.0
7,2008,IALI,126.900000,192.100000,196.100000,182.100000,24.100000,160.188235,48.235294,6.580000,...,66.0,68.0,70.0,76.0,305.0,235.0,160.0,107.0,42.0,19.0
8,2007,MNHE,77.252941,88.864706,123.582353,75.152941,94.541176,92.782353,45.311765,7.398235,...,70.0,72.0,76.0,78.0,422.0,331.0,258.0,143.0,105.0,75.0
9,2008,MNHE,81.500000,68.500000,100.800000,90.700000,66.000000,73.200000,59.623529,4.940000,...,70.0,72.0,76.0,78.0,422.0,331.0,258.0,143.0,105.0,75.0


In [24]:
env.shape

(1185, 86)

In [25]:
env.columns

Index(['YEAR', 'LOC', 'X04_PRCP', 'X05_PRCP', 'X06_PRCP', 'X07_PRCP',
       'X08_PRCP', 'X09_PRCP', 'X10_PRCP', 'X04_TAVG', 'X05_TAVG', 'X06_TAVG',
       'X07_TAVG', 'X08_TAVG', 'X09_TAVG', 'X10_TAVG', 'X04_DP01', 'X05_DP01',
       'X06_DP01', 'X07_DP01', 'X08_DP01', 'X09_DP01', 'X10_DP01', 'X04_DP10',
       'X05_DP10', 'X06_DP10', 'X07_DP10', 'X08_DP10', 'X09_DP10', 'X10_DP10',
       'X04_HTDD', 'X05_HTDD', 'X06_HTDD', 'X07_HTDD', 'X08_HTDD', 'X09_HTDD',
       'X10_HTDD', 'X04_CLDD', 'X05_CLDD', 'X06_CLDD', 'X07_CLDD', 'X08_CLDD',
       'X09_CLDD', 'X10_CLDD', 'cfvo_0_5cm', 'cfvo_5_15cm', 'cfvo_15_30cm',
       'cfvo_30_60cm', 'cfvo_60_100cm', 'cfvo_100_200cm', 'clay_0_5cm',
       'clay_5_15cm', 'clay_15_30cm', 'clay_30_60cm', 'clay_60_100cm',
       'clay_100_200cm', 'silt_0_5cm', 'silt_5_15cm', 'silt_15_30cm',
       'silt_30_60cm', 'silt_60_100cm', 'silt_100_200cm', 'nitrogen_0_5cm',
       'nitrogen_5_15cm', 'nitrogen_15_30cm', 'nitrogen_30_60cm',
       'nitrogen_60_100cm

In [37]:
# WEATHER: aggregate Apr–Oct 
months = ["04","05","06","07","08","09","10"]

def cols(prefix):
    return [f"X{m}_{prefix}" for m in months if f"X{m}_{prefix}" in env.columns]

weather_defs = {
    "PRCP_sum": ("PRCP", "sum"),
    "DP01_sum": ("DP01", "sum"),
    "DP10_sum": ("DP10", "sum"),
    "HTDD_sum": ("HTDD", "sum"),
    "CLDD_sum": ("CLDD", "sum"),
    "TAVG_mean": ("TAVG", "mean"),
}

weather_agg = env[["YEAR","LOC", 'cfvo_0_5cm', 'cfvo_5_15cm', 'cfvo_15_30cm',
       'cfvo_30_60cm', 'cfvo_60_100cm', 'cfvo_100_200cm', 'clay_0_5cm',
       'clay_5_15cm', 'clay_15_30cm', 'clay_30_60cm', 'clay_60_100cm',
       'clay_100_200cm', 'silt_0_5cm', 'silt_5_15cm', 'silt_15_30cm',
       'silt_30_60cm', 'silt_60_100cm', 'silt_100_200cm', 'nitrogen_0_5cm',
       'nitrogen_5_15cm', 'nitrogen_15_30cm', 'nitrogen_30_60cm',
       'nitrogen_60_100cm', 'nitrogen_100_200cm', 'sand_0_5cm', 'sand_5_15cm',
       'sand_15_30cm', 'sand_30_60cm', 'sand_60_100cm', 'sand_100_200cm',
       'phh2o_0_5cm', 'phh2o_5_15cm', 'phh2o_15_30cm', 'phh2o_30_60cm',
       'phh2o_60_100cm', 'phh2o_100_200cm', 'soc_0_5cm', 'soc_5_15cm',
       'soc_15_30cm', 'soc_30_60cm', 'soc_60_100cm', 'soc_100_200cm']].copy()

for new_name, (base, how) in weather_defs.items():
    use_cols = cols(base)
    if not use_cols:
        # column family missing; create NaN
        weather_agg[new_name] = pd.NA
        continue
    if how == "sum":
        weather_agg[new_name] = env[use_cols].sum(axis=1, min_count=1)
    else:
        weather_agg[new_name] = env[use_cols].mean(axis=1)

weather_agg.head()

,YEAR,LOC,cfvo_0_5cm,cfvo_5_15cm,cfvo_15_30cm,cfvo_30_60cm,cfvo_60_100cm,cfvo_100_200cm,clay_0_5cm,clay_5_15cm,...,soc_15_30cm,soc_30_60cm,soc_60_100cm,soc_100_200cm,PRCP_sum,DP01_sum,DP10_sum,HTDD_sum,CLDD_sum,TAVG_mean
0,2001,SDEP,5.0,3.0,3.0,8.0,12.0,21.0,388.0,393.0,...,124.0,74.0,46.0,27.0,484.800000,55.000000,39.000000,726.958824,459.564706,17.067983
1,2002,SDEP,5.0,3.0,3.0,8.0,12.0,21.0,388.0,393.0,...,124.0,74.0,46.0,27.0,525.400000,56.000000,43.000000,701.023529,530.070588,17.446891
2,2003,SDEP,5.0,3.0,3.0,8.0,12.0,21.0,388.0,393.0,...,124.0,74.0,46.0,27.0,602.800000,40.000000,32.000000,699.964706,464.211765,17.278319
3,2004,SDEP,5.0,3.0,3.0,8.0,12.0,21.0,388.0,393.0,...,124.0,74.0,46.0,27.0,552.541176,47.470588,37.176471,713.705882,441.211765,16.997647
4,2005,SDEP,5.0,3.0,3.0,8.0,12.0,21.0,388.0,393.0,...,124.0,74.0,46.0,27.0,678.600000,51.000000,43.000000,650.294118,483.658824,17.633277


In [38]:
weather_agg.shape

(1185, 50)

### Loading Phenotypic data

In [46]:
phen = pd.read_csv('/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/C1_Phenotype_Data_V2.csv')
phen.head(10)

/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/4056204764.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  phen = pd.read_csv('/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/C1_Phenotype_Data_V2.csv')


,Unnamed: 0,projects_x,ProjectID,shorthand_x,YEAR_x,LOC,LONGITUDE,LATITUDE,LINE,ERM,...,shorthand_y,Unnamed: 0_y,MAB_PROJECT_ID,YEAR_y,GERMPLASM_ID,GENERATION_NAME,HG,FILE_LIST,CROSS,GERMPLASM_ID_TESTER
0,0,project209265_year2001.Rdata,209265,C1.1,2001,NEDA,-96.41,42.42,191,NaN,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
1,1,project209265_year2001.Rdata,209265,C1.1,2001,NEDA,-96.41,42.42,193,NaN,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
2,2,project209265_year2001.Rdata,209265,C1.1,2001,IAPR,-95.62,43.08,62,102.066,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
3,3,project209265_year2001.Rdata,209265,C1.1,2001,IADA,-94.06,42.26,160,104.683,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
4,4,project209265_year2001.Rdata,209265,C1.1,2001,IAPR,-95.62,43.08,61,102.430,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
5,5,project209265_year2001.Rdata,209265,C1.1,2001,IAPR,-95.62,43.08,165,106.069,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
6,6,project209265_year2001.Rdata,209265,C1.1,2001,NEDA,-96.41,42.42,20,NaN,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
7,7,project209265_year2001.Rdata,209265,C1.1,2001,IAFO,-95.15,43.24,107,103.040,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
8,8,project209265_year2001.Rdata,209265,C1.1,2001,MNOW,-93.22,44.01,148,NaN,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
9,9,project209265_year2001.Rdata,209265,C1.1,2001,IADA,-94.06,42.26,114,105.851,...,C1.1,1311,209265,2001,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0


In [40]:
phen.shape

(536936, 33)

In [41]:
phen.columns

Index(['Unnamed: 0', 'projects_x', 'ProjectID', 'shorthand_x', 'YEAR_x', 'LOC',
       'LONGITUDE', 'LATITUDE', 'LINE', 'ERM', 'MST', 'PHT', 'RTLP', 'STLP',
       'TWT', 'YLD_BE', 'EHT', 'SET', 'CLUSTER', 'LINE_UNIQUE_ID',
       'Unnamed: 0_x', 'projects_y', 'projectID', 'shorthand_y',
       'Unnamed: 0_y', 'MAB_PROJECT_ID', 'YEAR_y', 'GERMPLASM_ID',
       'GENERATION_NAME', 'HG', 'FILE_LIST', 'CROSS', 'GERMPLASM_ID_TESTER'],
      dtype='object')

In [47]:
# drop helper columns like 'Unnamed: 0'
phen = phen.loc[:, ~phen.columns.str.match(r"^Unnamed")]
phen.shape

(536936, 30)

In [48]:
phen.columns

Index(['projects_x', 'ProjectID', 'shorthand_x', 'YEAR_x', 'LOC', 'LONGITUDE',
       'LATITUDE', 'LINE', 'ERM', 'MST', 'PHT', 'RTLP', 'STLP', 'TWT',
       'YLD_BE', 'EHT', 'SET', 'CLUSTER', 'LINE_UNIQUE_ID', 'projects_y',
       'projectID', 'shorthand_y', 'MAB_PROJECT_ID', 'YEAR_y', 'GERMPLASM_ID',
       'GENERATION_NAME', 'HG', 'FILE_LIST', 'CROSS', 'GERMPLASM_ID_TESTER'],
      dtype='object')

In [49]:
phen = phen.drop(columns=[
    'YEAR_y', 'projects_y', 'shorthand_y', 'projects_x', 'shorthand_x'
    ]).rename(columns={'YEAR_x': 'YEAR'})
phen.head()

,ProjectID,YEAR,LOC,LONGITUDE,LATITUDE,LINE,ERM,MST,PHT,RTLP,...,CLUSTER,LINE_UNIQUE_ID,projectID,MAB_PROJECT_ID,GERMPLASM_ID,GENERATION_NAME,HG,FILE_LIST,CROSS,GERMPLASM_ID_TESTER
0,209265,2001,NEDA,-96.41,42.42,191,NaN,19.7,89.0,NaN,...,1,C1.1.191,209265,209265,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
1,209265,2001,NEDA,-96.41,42.42,193,NaN,20.2,85.0,NaN,...,1,C1.1.193,209265,209265,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
2,209265,2001,IAPR,-95.62,43.08,62,102.066,19.1,83.0,NaN,...,1,C1.1.62,209265,209265,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
3,209265,2001,IADA,-94.06,42.26,160,104.683,22.8,NaN,0.0,...,1,C1.1.160,209265,209265,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0
4,209265,2001,IAPR,-95.62,43.08,61,102.430,19.3,80.0,NaN,...,1,C1.1.61,209265,209265,1589589,F2,Cluster1,project209265_year2001.Rdata,1589589/200761,1376340.0


In [51]:
# --- normalize LOC ---
phen["LOC"] = phen["LOC"].astype(str).str.strip().str.upper()

# --- basic type fixes ---
num_cols = ["LONGITUDE","LATITUDE","ERM","MST","PHT","RTLP","STLP","TWT","YLD_BE","EHT"]
for c in num_cols:
    if c in phen.columns:
        phen[c] = pd.to_numeric(phen[c], errors="coerce")

# cluster as int/categorical
if "CLUSTER" in phen.columns:
    phen["CLUSTER"] = pd.to_numeric(phen["CLUSTER"], errors="coerce").astype("Int64")

In [52]:
phen["LINE_UNIQUE_ID"] = phen["LINE_UNIQUE_ID"].astype(str).str.strip()

In [ ]:
# derive population/line numbers (handy later for genomic join)
m = phen["LINE_UNIQUE_ID"].str.extract(r"^C(?P<cluster>\d)\.(?P<population>\d+)\.(?P<line>\d+)$")
for k in ["cluster","population","line"]:
    phen[k] = pd.to_numeric(m[k], errors="coerce").astype("Int64")

In [54]:
# select core columns in a stable order 
core_cols = [
    "YEAR","LOC","LONGITUDE","LATITUDE",
    "CLUSTER","LINE_UNIQUE_ID","LINE","SET",
    "ERM","MST","PHT","RTLP","STLP","TWT","YLD_BE","EHT",
    "population","line"  # derived helpers
]
keep = [c for c in core_cols if c in phen.columns]
phen_clean = phen[keep].copy()

In [ ]:
# drop rows missing essential keys
phen_clean = phen_clean.dropna(subset=["YEAR","LOC","LINE_UNIQUE_ID"])

# YEAR integer if safe
phen_clean["YEAR"] = pd.to_numeric(phen_clean["YEAR"], errors="coerce").astype("Int64")

In [56]:
# deduplicate (conservative): keep first per YEAR-LOC-LINE_UNIQUE_ID-SET
dedup_keys = [k for k in ["YEAR","LOC","LINE_UNIQUE_ID","SET"] if k in phen_clean.columns]
phen_clean = phen_clean.sort_index().drop_duplicates(subset=dedup_keys, keep="first")

In [58]:
phen_clean.shape

(535474, 18)

In [59]:
# Merge Phenotype and environment data
df_phen_env = phen_clean.merge(
    weather_agg,
    on=['YEAR', 'LOC'],
    how='left'
)
df_phen_env.head()

,YEAR,LOC,LONGITUDE,LATITUDE,CLUSTER,LINE_UNIQUE_ID,LINE,SET,ERM,MST,...,soc_15_30cm,soc_30_60cm,soc_60_100cm,soc_100_200cm,PRCP_sum,DP01_sum,DP10_sum,HTDD_sum,CLDD_sum,TAVG_mean
0,2001,NEDA,-96.41,42.42,1,C1.1.191,191,0303300303Z0,NaN,19.7,...,120.0,77.0,54.0,38.0,658.729412,71.823529,48.000000,664.141177,519.935294,17.822605
1,2001,NEDA,-96.41,42.42,1,C1.1.193,193,0303300303Z0,NaN,20.2,...,120.0,77.0,54.0,38.0,658.729412,71.823529,48.000000,664.141177,519.935294,17.822605
2,2001,IAPR,-95.62,43.08,1,C1.1.62,62,0303300303Z0,102.066,19.1,...,159.0,90.0,37.0,17.0,700.735294,52.647059,40.882353,710.211765,403.594118,16.773529
3,2001,IADA,-94.06,42.26,1,C1.1.160,160,0303300303Z0,104.683,22.8,...,143.0,83.0,43.0,20.0,695.294118,60.705882,41.882353,630.376471,474.547059,17.588655
4,2001,IAPR,-95.62,43.08,1,C1.1.61,61,0303300303Z0,102.430,19.3,...,159.0,90.0,37.0,17.0,700.735294,52.647059,40.882353,710.211765,403.594118,16.773529


In [60]:
df_phen_env.shape

(535474, 66)

In [61]:
df_phen_env.columns

Index(['YEAR', 'LOC', 'LONGITUDE', 'LATITUDE', 'CLUSTER', 'LINE_UNIQUE_ID',
       'LINE', 'SET', 'ERM', 'MST', 'PHT', 'RTLP', 'STLP', 'TWT', 'YLD_BE',
       'EHT', 'population', 'line', 'cfvo_0_5cm', 'cfvo_5_15cm',
       'cfvo_15_30cm', 'cfvo_30_60cm', 'cfvo_60_100cm', 'cfvo_100_200cm',
       'clay_0_5cm', 'clay_5_15cm', 'clay_15_30cm', 'clay_30_60cm',
       'clay_60_100cm', 'clay_100_200cm', 'silt_0_5cm', 'silt_5_15cm',
       'silt_15_30cm', 'silt_30_60cm', 'silt_60_100cm', 'silt_100_200cm',
       'nitrogen_0_5cm', 'nitrogen_5_15cm', 'nitrogen_15_30cm',
       'nitrogen_30_60cm', 'nitrogen_60_100cm', 'nitrogen_100_200cm',
       'sand_0_5cm', 'sand_5_15cm', 'sand_15_30cm', 'sand_30_60cm',
       'sand_60_100cm', 'sand_100_200cm', 'phh2o_0_5cm', 'phh2o_5_15cm',
       'phh2o_15_30cm', 'phh2o_30_60cm', 'phh2o_60_100cm', 'phh2o_100_200cm',
       'soc_0_5cm', 'soc_5_15cm', 'soc_15_30cm', 'soc_30_60cm', 'soc_60_100cm',
       'soc_100_200cm', 'PRCP_sum', 'DP01_sum', 'DP10_sum', 'HT

In [65]:
df_phen_env['LINE'].unique()

array([191, 193, 62, ..., 1201, 1013, 1170], shape=(2330,), dtype=object)

In [100]:
df_phen_env['population'].unique()

<IntegerArray>
[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
 ...
 491, 492, 493, 494, 495, 496, 497, 498, 499, 500]
Length: 496, dtype: Int64

In [74]:
df_phen_env['YLD_BE'].isna().sum()

np.int64(20651)

### Genotype data

In [62]:
gen = pd.read_csv('/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/C1.1_Imputed.csv')
gen.head()

,Unnamed: 0,M00003409443,M00000005000,M00000226152,M00000038753,M00009474085,M00003426327,M00009437705,M00009450083,M00003679000,...,M00003426049,M00003175971,M00003149874,M00003344384,M00009474181,M00000423059,M00003426582,M00009451648,M00009451668,M00009474563
0,PID200761,1,1,1,-1,1.0,-1,1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
1,PID1589589,1,1,1,-1,-1.0,-1,1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
2,00000000001,1,1,1,-1,NaN,-1,1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
3,00000000002,1,1,1,-1,NaN,-1,1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
4,00000000003,1,1,1,-1,NaN,-1,1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1


In [63]:
gen.shape

(182, 2912)

In [81]:
gen_miss = (gen.isna().mean()*100).sort_values(ascending=False)
gen_miss.tail()

M00003298131    0.0
M00003286674    0.0
M00000016503    0.0
M00009474896    0.0
M00009474563    0.0
dtype: float64

In [84]:
from pathlib import Path

In [86]:
# ---- Settings ----
DATA_DIR = Path("/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1")
PATTERN = "C1.*_Imputed.csv"  # e.g., C1.1_Imputed.csv ... C1.500_Imputed.csv
MARKER_REGEX = re.compile(r"^M\d+")  # keep columns that look like 'M0000...'


def extract_marker_columns(csv_path: Path) -> set:
    """
    Read only the header of a CSV and return the set of marker columns.
    We define marker columns as those starting with 'M' followed by digits.
    """
    cols = pd.read_csv(csv_path, nrows=0).columns
    markers = {c for c in cols if MARKER_REGEX.match(str(c))}
    return markers


# ---- Discover files ----
files = sorted(DATA_DIR.glob(PATTERN), key=lambda p: (
    int(p.name.split("_")[0].split(".")[1]) if "." in p.name else 0
))

if not files:
    raise FileNotFoundError(f"No files matched {DATA_DIR / PATTERN}")

# ---- Collect marker sets ----
marker_sets = {}
for f in files:
    try:
        marker_sets[f.name] = extract_marker_columns(f)
    except Exception as e:
        marker_sets[f.name] = set()
        print(f"Warning: failed to read {f.name}: {e}")

# ---- Compute intersection across all populations ----
all_files = list(marker_sets.keys())
common_markers = set.intersection(*(s for s in marker_sets.values() if len(s) > 0))

# ---- Build a summary table ----
summary_rows = []
for fname, markers in marker_sets.items():
    summary_rows.append({
        "file": fname,
        "n_markers_in_file": len(markers),
        "n_common_markers_so_far": len(common_markers),
    })
summary = pd.DataFrame(summary_rows)

# ---- Persist outputs ----
common_txt = DATA_DIR / "C1_all_pops_common_markers.txt"
counts_csv = DATA_DIR / "C1_marker_counts_summary.csv"
summary.to_csv(counts_csv, index=False)
pd.Series(sorted(common_markers)).to_csv(common_txt, index=False, header=False)

# ---- Display results ----

print(f"Files found: {len(files)}")
print(f"Common markers across ALL matched files: {len(common_markers)}")
print(f"Saved common marker list to: {common_txt}")
print(f"Saved per-file counts to: {counts_csv}")


Files found: 499
Common markers across ALL matched files: 2911
Saved common marker list to: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/C1_all_pops_common_markers.txt
Saved per-file counts to: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/C1_marker_counts_summary.csv


In [87]:
summary

,file,n_markers_in_file,n_common_markers_so_far
0,C1.1_Imputed.csv,2911,2911
1,C1.2_Imputed.csv,2911,2911
2,C1.3_Imputed.csv,2911,2911
3,C1.4_Imputed.csv,2911,2911
4,C1.5_Imputed.csv,2911,2911
...,...,...,...
494,C1.496_Imputed.csv,2911,2911
495,C1.497_Imputed.csv,2911,2911
496,C1.498_Imputed.csv,2911,2911
497,C1.499_Imputed.csv,2911,2911


In [88]:
# ---- Config ----
DATA_DIR = Path("/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1")
PATTERN = "C1.*_Imputed.csv"          # expects files like C1.1_Imputed.csv ... C1.500_Imputed.csv
MARKER_REGEX = re.compile(r"^M\d+")   # marker columns look like 'M0000...'
THRESH = 0.05                         # <=5% missingness threshold
OUT_DIR = DATA_DIR / "qc_markers_5pct"
OUT_DIR.mkdir(exist_ok=True)

def marker_columns(df: pd.DataFrame) -> list[str]:
    return [c for c in df.columns if MARKER_REGEX.match(str(c))]

def compute_good_markers(csv_path: Path, thresh: float = THRESH) -> dict:
    """Return counts and list of markers passing missingness threshold for one file."""
    df = pd.read_csv(csv_path)
    mcols = marker_columns(df)
    miss = df[mcols].isna().mean()
    good = miss[miss <= thresh].index.tolist()
    # Persist missingness (optional, for debugging)
    miss_path = OUT_DIR / f"{csv_path.stem}_missingness.csv"
    miss.sort_values(ascending=False).to_csv(miss_path, header=["missing_rate"])
    return {
        "file": csv_path.name,
        "n_lines": df.shape[0],
        "n_markers_total": len(mcols),
        "n_markers_good": len(good),
        "good_markers": good,
    }

# ---- Discover files ----
files = sorted(DATA_DIR.glob(PATTERN), key=lambda p: (
    int(p.name.split("_")[0].split(".")[1]) if "." in p.name else 0
))

if not files:
    raise FileNotFoundError(f"No files matched {DATA_DIR / PATTERN}")

# ---- Per-file computation ----
rows = []
good_sets = []
for f in files:
    res = compute_good_markers(f, THRESH)
    rows.append({k: v for k, v in res.items() if k != "good_markers"})
    good_sets.append(set(res["good_markers"]))
    # Save per-file good markers
    out_list = OUT_DIR / f"{f.stem}_good_markers.txt"
    pd.Series(sorted(res["good_markers"])).to_csv(out_list, index=False, header=False)

# Summary table
summary = pd.DataFrame(rows)
summary_path = OUT_DIR / "per_file_qc_summary.csv"
summary.to_csv(summary_path, index=False)

# ---- Intersection across ALL pops of 'good' markers ----
if len(good_sets) == 1:
    common_good = good_sets[0]
else:
    common_good = set.intersection(*good_sets) if all(len(s) > 0 for s in good_sets) else set()

common_path = OUT_DIR / "C1_good_markers_common_5pct.txt"
pd.Series(sorted(common_good)).to_csv(common_path, index=False, header=False)


print(f"Files found: {len(files)}")
print(f"Saved per-file QC summary: {summary_path}")
print(f"Markers common & <=5% missing across ALL: {len(common_good)}")
print(f"Saved common-good marker list: {common_path}")


/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/1196077801.py:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/1196077801.py:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Files found: 499
Saved per-file QC summary: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/qc_markers_5pct/per_file_qc_summary.csv
Markers common & <=5% missing across ALL: 6
Saved common-good marker list: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/qc_markers_5pct/C1_good_markers_common_5pct.txt


In [91]:
summary['n_markers_good'].min()

np.int64(1491)

In [92]:
summary['n_markers_good'].max()

np.int64(2671)

In [93]:
# Compute per-marker "complete presence" counts across populations

# ---- Config ----
DATA_DIR = Path("/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1")
PATTERN = "C1.*_Imputed.csv"            # e.g., C1.1_Imputed.csv ... C1.500_Imputed.csv
MARKER_REGEX = re.compile(r"^M\d+")     # columns like 'M0000...'
OUT_DIR = DATA_DIR / "marker_presence"
OUT_DIR.mkdir(exist_ok=True)

# ---- Discover files ----
files = sorted(DATA_DIR.glob(PATTERN), key=lambda p: (
    int(p.name.split("_")[0].split(".")[1]) if "." in p.name else 0
))

if not files:
    raise FileNotFoundError(f"No files matched {DATA_DIR / PATTERN}")

# ---- First pass: collect marker columns per file & union ----
file_markers = {}
all_markers = set()

for f in files:
    cols = pd.read_csv(f, nrows=0).columns
    mcols = [c for c in cols if MARKER_REGEX.match(str(c))]
    file_markers[f] = mcols
    all_markers.update(mcols)

# ---- Initialize presence counts ----
presence_counts = {m: 0 for m in all_markers}
total_pops = len(files)

# ---- Second pass: count "completely present" (no NaNs) per marker per file ----
for f in files:
    mcols = file_markers[f]
    if not mcols:
        continue
    df = pd.read_csv(f, usecols=mcols)       # read only marker columns
    complete = (df.isna().sum(axis=0) == 0)  # True if no NaNs in that population for the marker
    for m, ok in complete.items():
        if ok:
            presence_counts[m] += 1

# ---- Build results ----
presence_df = (
    pd.DataFrame({"marker": list(presence_counts.keys()),
                  "present_in_pops": list(presence_counts.values())})
    .assign(total_pops=total_pops)
    .assign(frac_present=lambda d: d["present_in_pops"] / d["total_pops"])
    .sort_values(["present_in_pops", "marker"], ascending=[False, True])
    .reset_index(drop=True)
)

# Save full table
presence_path = OUT_DIR / "marker_presence_counts.csv"
presence_df.to_csv(presence_path, index=False)

# Summary for thresholds the user asked about
thresholds = [400, 450]
summary = {f">={k}": int((presence_df["present_in_pops"] >= k).sum()) for k in thresholds}

# Distribution table (how many markers have exactly k populations with complete presence)
dist = presence_df["present_in_pops"].value_counts().sort_index()
dist_df = dist.rename_axis("present_in_pops").reset_index(name="n_markers")
dist_path = OUT_DIR / "marker_presence_distribution.csv"
dist_df.to_csv(dist_path, index=False)

print(f"Files considered: {total_pops}")
print("Counts of markers with complete presence at thresholds:", summary)
print(f"Saved full presence table: {presence_path}")
print(f"Saved distribution table: {dist_path}")


Files considered: 499
Counts of markers with complete presence at thresholds: {'>=400': 783, '>=450': 361}
Saved full presence table: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/marker_presence/marker_presence_counts.csv
Saved distribution table: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/marker_presence/marker_presence_distribution.csv


### global ≤5% missing

In [ ]:
# Re-run with careful state (kernel was reset). Same as previous cell.
from collections import defaultdict
from typing import List, Dict
import matplotlib.pyplot as plt

# ---- Config ----
DATA_DIR = Path("/Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1")
PATTERN = "C1.*_Imputed.csv"
ID_COL = "Unnamed: 0"              # line identifier found in these files
MARKER_RE = re.compile(r"^M\d+")   # marker columns look like 'M0000...'
MISS_THR = 0.1
MAX_PCS = 50
OUT = DATA_DIR / "pca_global5"
OUT.mkdir(exist_ok=True)

# ---- Discover files ----
files = sorted(DATA_DIR.glob(PATTERN), key=lambda p: int(p.name.split("_")[0].split(".")[1]))
if not files:
    raise FileNotFoundError(f"No files matched {DATA_DIR / PATTERN}")

# ---- Pass 0: get union of markers across headers ----
all_markers = set()
file_markers: Dict[Path, List[str]] = {}
for fp in files:
    cols = pd.read_csv(fp, nrows=0).columns
    mcols = [c for c in cols if MARKER_RE.match(str(c))]
    file_markers[fp] = mcols
    all_markers.update(mcols)

all_markers = sorted(all_markers)
pd.Series(all_markers).to_csv(OUT/"union_marker_ids.txt", index=False, header=False)

# ---- Pass 1: compute global missing counts with streaming ----
missing_counts = defaultdict(int)
total_counts   = defaultdict(int)

for fp in files:
    # number of lines (rows) in this file
    n_rows = len(pd.read_csv(fp, usecols=[ID_COL]))
    present = set(file_markers[fp])
    absent = set(all_markers) - present
    # For present markers: read them and count NaNs
    if present:
        dfm = pd.read_csv(fp, usecols=list(present))
        na_sum = dfm.isna().sum(axis=0)  # Series indexed by marker
        for m, s in na_sum.items():
            missing_counts[m] += int(s)
            total_counts[m]   += n_rows
    # For absent markers: count entire column as missing for these rows
    for m in absent:
        missing_counts[m] += n_rows
        total_counts[m]   += n_rows

# Build missing rate table
miss_rate = pd.Series({m: (missing_counts[m] / total_counts[m]) for m in all_markers}).sort_index()
miss_tbl = pd.DataFrame({"marker": miss_rate.index, "missing_rate": miss_rate.values}).sort_values("missing_rate")
miss_tbl.to_csv(OUT/"marker_global_missing_rates.csv", index=False)

# Select markers with global missing ≤ 5%
keep_markers = miss_tbl.loc[miss_tbl["missing_rate"] <= MISS_THR, "marker"].tolist()
pd.Series(keep_markers).to_csv(OUT/"markers_global_missing_le5pct.txt", index=False, header=False)

print(f"Files found: {len(files)}")
print(f"Union markers: {len(all_markers)}")
print(f"Markers with global missing ≤{int(MISS_THR*100)}%: {len(keep_markers)}")

# ---- Pass 2: build stacked dataset with only kept markers + IDs + population ----
stack_parts = []
for fp in files:
    pop_name = fp.stem.split("_")[0]   # e.g., "C1.1"
    # read ID + markers present in this file that are also in keep list
    present_kept = sorted(set(file_markers[fp]).intersection(keep_markers))
    usecols = [ID_COL] + present_kept
    df = pd.read_csv(fp, usecols=usecols)
    # ensure all kept markers are present; if some are absent in this file, add as NaN
    for m in keep_markers:
        if m not in df.columns:
            df[m] = np.nan
    # reorder columns: ID, markers in keep_markers order
    df = df[[ID_COL] + keep_markers]
    df.insert(1, "population", pop_name)
    stack_parts.append(df)

stack = pd.concat(stack_parts, axis=0, ignore_index=True)
stack.to_csv(OUT/"stack_kept_markers_raw.csv", index=False)

print(f"Stacked matrix shape (raw, with NaNs): {stack.shape}")




/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/2826073579.py:39: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  n_rows = len(pd.read_csv(fp, usecols=[ID_COL]))
/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/2826073579.py:39: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  n_rows = len(pd.read_csv(fp, usecols=[ID_COL]))


Files found: 499
Union markers: 2911
Markers with global missing ≤10%: 1077


/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/2826073579.py:74: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp, usecols=usecols)
/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_7834/2826073579.py:74: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp, usecols=usecols)


Stacked matrix shape (raw, with NaNs): (78396, 1079)


In [96]:
# ---- Pass 3: per-pop mode imputation for -1/0/1 markers ----
def series_mode_ignore_na(s: pd.Series):
    m = s.mode(dropna=True)
    return m.iloc[0] if len(m) else np.nan

imputed_parts = []
for pop, sub in stack.groupby("population", sort=False):
    X = sub[keep_markers]
    # compute per-marker mode within this population
    modes = X.apply(series_mode_ignore_na, axis=0)
    # if a marker's mode is NaN for this pop (all NaN), fallback to global mode across all lines
    global_modes = stack[keep_markers].apply(series_mode_ignore_na, axis=0)
    fill_vals = modes.fillna(global_modes)
    X_imp = X.fillna(fill_vals)
    out = pd.concat([sub[[ID_COL, "population"]].reset_index(drop=True), X_imp.reset_index(drop=True)], axis=1)
    imputed_parts.append(out)

imputed = pd.concat(imputed_parts, axis=0, ignore_index=True)
imputed.to_csv(OUT/"stack_kept_markers_imputed.csv", index=False)

# ---- Pass 4: standardize markers globally (mean 0, var 1)
# convert to float for scaling
X = imputed[keep_markers].astype(float)
means = X.mean(axis=0)
stds = X.std(axis=0, ddof=0).replace(0.0, 1.0)  # guard against constant columns
Xz = (X - means) / stds

means.to_csv(OUT/"scaler_means.csv", header=["mean"])
stds.to_csv(OUT/"scaler_stds.csv", header=["std"])

# ---- Pass 5: PCA (fit up to MAX_PCS)
from sklearn.decomposition import PCA

n_components = min(MAX_PCS, Xz.shape[1], Xz.shape[0])
pca = PCA(n_components=n_components, svd_solver="auto", random_state=42)
scores = pca.fit_transform(Xz.values)  # shape: (n_samples, n_components)

# explained variance info
evr = pca.explained_variance_ratio_
cum_evr = np.cumsum(evr)
evr_df = pd.DataFrame({"pc": np.arange(1, n_components+1), "explained_variance_ratio": evr, "cumulative": cum_evr})
evr_df.to_csv(OUT/"pca_explained_variance.csv", index=False)

# choose k = smallest number of PCs to reach 90% variance (or max available)
k90 = int(np.searchsorted(cum_evr, 0.90, side="left") + 1)
k90 = min(k90, n_components)
scores_k = scores[:, :k90]

# save scores with IDs
scores_df = pd.DataFrame(scores_k, columns=[f"PC{i}" for i in range(1, k90+1)])
scores_df.insert(0, "population", imputed["population"].values)
scores_df.insert(0, ID_COL, imputed[ID_COL].values)
scores_df.to_csv(OUT/"global_pca_scores_k90.csv", index=False)

# ---- Plot cumulative explained variance
plt.figure()
plt.plot(evr_df["pc"], evr_df["cumulative"])
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.title("Global PCA (stacked) — cumulative explained variance")
plt.tight_layout()
plot_path = OUT/"pca_cumulative_variance.png"
plt.savefig(plot_path)
plt.close()

print(f"PCA components fit: {n_components}")
print(f"PCs to reach ~90% variance: k={k90}")
print("Saved outputs to:", str(OUT))

PCA components fit: 50
PCs to reach ~90% variance: k=50
Saved outputs to: /Users/gurjitsingh/Desktop/MS Data Science/UGA_Bayer_Hackathon_2025/ImputedPopulationsC1/pca_global5


In [129]:
# === 0) Start from the cleaned frames you already made ===
geno  = gen_pca_imputed.copy()
pheno = df_phen_env.copy()

# -- PHENO: drop NA keys, coerce, and build "C1.x"
for c in ["population", "line"]:
    pheno[c] = pd.to_numeric(pheno[c], errors="coerce")
pheno_clean = pheno.dropna(subset=["population", "line"]).copy()
pheno_clean["population"] = pheno_clean["population"].astype("int64")
pheno_clean["line"]       = pheno_clean["line"].astype("int64")
pheno_clean["population_str"] = "C1." + pheno_clean["population"].astype(str)

# -- GENO: coerce line from Unnamed: 0; drop parents; make population_str
geno = geno.rename(columns={"Unnamed: 0": "raw_id"})
geno["line"] = pd.to_numeric(geno["raw_id"], errors="coerce")
geno_nopar = geno.loc[geno["line"].notna()].copy()
geno_nopar["line"] = geno_nopar["line"].astype("int64")
geno_nopar["population_str"] = geno_nopar["population"].astype(str)

# === 1) Diagnose duplicates on the right keys ===
keys = ["population_str", "line"]
dup_mask = geno_nopar.duplicated(subset=keys, keep=False)
print("Duplicate (population,line) rows in GENO:", dup_mask.sum())

# Any duplicate groups that actually differ in values?
value_cols = [c for c in geno_nopar.columns if c not in {"raw_id"}]
vary_groups = (
    geno_nopar.loc[dup_mask]
    .groupby(keys)[value_cols]
    .nunique(dropna=False)
    .max(axis=1) > 1
)
print("Duplicate groups with conflicting values:", vary_groups.sum())

# === 2) Resolve duplicates ===
def safe_mode(s: pd.Series):
    m = s.mode(dropna=True)
    if not m.empty:
        return m.iloc[0]
    # fallbacks if all NA or no mode
    return s.dropna().iloc[0] if s.notna().any() else pd.NA

if vary_groups.sum() == 0:
    # Duplicates are identical across value columns -> just drop them
    geno_dedup = geno_nopar.drop_duplicates(subset=keys, keep="first")
else:
    # Mixed duplicates: use mean for floats (PCs), mode for integer markers, mode for objects
    agg_dict = {}
    for c in value_cols:
        if c in keys:  # groupby will re-add keys for us
            continue
        if pd.api.types.is_float_dtype(geno_nopar[c]):
            agg_dict[c] = "mean"
        elif pd.api.types.is_numeric_dtype(geno_nopar[c]):
            agg_dict[c] = safe_mode
        else:
            agg_dict[c] = safe_mode
    geno_dedup = geno_nopar.groupby(keys, as_index=False).agg(agg_dict)

print("Genotype rows after de-dup:", len(geno_dedup))

# === 3) Merge (many pheno to one geno) ===
merged_all_markers = pheno_clean.merge(
    geno_dedup,
    on=keys,
    how="left",
    validate="m:1")


Duplicate (population,line) rows in GENO: 100
Duplicate groups with conflicting values: 50
Genotype rows after de-dup: 76995


In [131]:
merged_all_markers.head()

,YEAR,LOC,LONGITUDE,LATITUDE,CLUSTER,LINE_UNIQUE_ID,LINE,SET,ERM,MST,...,M00003215685,M00000038964,M00009449568,M00003223925,M00009450576,M00000596277,M00003306099,M00000627733,M00009451825,M00009450073
0,2001,NEDA,-96.41,42.42,1,C1.1.191,191,0303300303Z0,NaN,19.7,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,0.0
1,2001,NEDA,-96.41,42.42,1,C1.1.193,193,0303300303Z0,NaN,20.2,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,0.0
2,2001,IAPR,-95.62,43.08,1,C1.1.62,62,0303300303Z0,102.066,19.1,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,0.0
3,2001,IADA,-94.06,42.26,1,C1.1.160,160,0303300303Z0,104.683,22.8,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
4,2001,IAPR,-95.62,43.08,1,C1.1.61,61,0303300303Z0,102.430,19.3,...,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,0.0


In [135]:
merged_all_markers.to_csv('merged_all_markers.csv', index=False)

In [126]:
# ---------- Inputs ----------

scores_df = pd.read_csv(OUT / "global_pca_scores_k90.csv")  # has [ID_COL, population, PC1..PCk]

ID_COL = scores_df.columns[0]  # first column is the line identifier you saved

# ---------- Phenotype/env: clean + keys ----------
pheno = df_phen_env.copy()

# (optional but sensible if your PCs are for C1) — filter to C1 only
# pheno = pheno.loc[pheno["CLUSTER"].astype(str).str.upper().eq("C1")].copy()

for c in ["population", "line"]:
    pheno[c] = pd.to_numeric(pheno[c], errors="coerce")

pheno_clean = pheno.dropna(subset=["population", "line"]).copy()
pheno_clean["population"] = pheno_clean["population"].astype("int64")
pheno_clean["line"]       = pheno_clean["line"].astype("int64")
pheno_clean["population_str"] = "C1." + pheno_clean["population"].astype(str)

# ---------- Geno PCs: coerce ID to numeric line; make keys ----------
pcs = scores_df.copy()
pcs = pcs.rename(columns={ID_COL: "raw_id"})
pcs["line"] = pd.to_numeric(pcs["raw_id"], errors="coerce")          # parents (PID...) -> NaN
pcs = pcs.loc[pcs["line"].notna()].copy()                             # drop parents
pcs["line"] = pcs["line"].astype("int64")
pcs["population_str"] = pcs["population"].astype(str)

pc_cols = [c for c in pcs.columns if c.startswith("PC")]              # only keep PCs + keys

# If there are duplicates per (population,line), average PCs
pcs_agg = pcs.groupby(["population_str", "line"], as_index=False)[pc_cols].mean()

# ---------- Merge (many pheno -> one geno) ----------
merged_50pc = pheno_clean.merge(
    pcs_agg,
    on=["population_str", "line"],
    how="left",
    validate="m:1"
)

# ---------- Quick audit ----------
n = len(merged_50pc)
hit = merged_50pc[pc_cols].notna().any(axis=1)
print(f"Matched PCA for {hit.sum():,} of {n:,} rows ({hit.mean():.1%}).")
print("PC columns attached:", len(pc_cols))


Matched PCA for 530,986 of 530,986 rows (100.0%).
PC columns attached: 50


In [ ]:
merged_50pc.head

,YEAR,LOC,LONGITUDE,LATITUDE,CLUSTER,LINE_UNIQUE_ID,LINE,SET,ERM,MST,...,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50
0,2001,NEDA,-96.41,42.42,1,C1.1.191,191,0303300303Z0,NaN,19.7,...,0.109421,-0.406759,2.648996,-1.380549,2.351164,1.903021,2.724588,-0.407803,1.070156,3.852213
1,2001,NEDA,-96.41,42.42,1,C1.1.193,193,0303300303Z0,NaN,20.2,...,-0.104107,-0.302986,3.104011,-1.343930,2.254312,1.847658,2.598561,0.070087,1.497785,4.229651
2,2001,IAPR,-95.62,43.08,1,C1.1.62,62,0303300303Z0,102.066,19.1,...,0.438589,-1.265800,3.438342,0.044496,2.238683,1.406091,2.587688,-1.387403,1.623629,3.461917
3,2001,IADA,-94.06,42.26,1,C1.1.160,160,0303300303Z0,104.683,22.8,...,-0.021742,0.702236,2.063786,-0.788070,1.928100,0.647655,2.147769,-0.623308,1.805511,3.242088
4,2001,IAPR,-95.62,43.08,1,C1.1.61,61,0303300303Z0,102.430,19.3,...,-0.091467,-0.231861,2.749915,-2.001866,2.252678,3.077253,2.915370,-1.262921,1.758283,4.299262


In [132]:
merged_50pc.shape

(530986, 117)

In [134]:
merged_50pc.to_csv('merged_50pc.csv', index=False)